# Requête HTTP 

Un requête HTTP est une requête basée sur le protocole TCP, elle fait partie de la couche application de la couche OSI. Elle permet d'accéder aux données mise à disposition sur une adresse IP (ou url résolue par un DNS) et un port. 

Les deux ports les plus utilisés dans le web sont le 80 pour les sites en HTTP et le 443 pour les sites en HTTPS. HTTPS est une variable du protocole HTTP basé sur le protocole TLS.

Il existe de nombreux types de requêtes selon la convention `REST`: 
- GET
- POST
- PUT 
- DELETE
- UPDATE.

Dans notre cas, nous allons utiliser la plupart du temps des GET et potentiellement des POST. 
- Le GET permet comme son nom l'indique de récupérer des informations en fonction de certains paramètres. 
- Le POST nécessite un envoi de données pour récupérer des données. Le body du post est, la plupart du temps, envoyé sous la forme d'un objet JSON.

Ces requêtes encapsulent un certain nombre de paramètres qui permettent soient d'identifier une provenance et un utilisateur ou de réaliser différentes actions.

In [1]:
import requests

In [2]:
url = "https://www.esiee.fr/"
response = requests.get(url)
response.status_code

200

Il existe deux méthodes pour récupérer le contenu de la page :

- `response.text` qui permet de retourner le texte sous la forme d'une chaine de charactères.
- `response.content` qui permet de récupérer le contenu de la page sous la forme de bytes

In [3]:
type(response.content)

bytes

In [4]:
type(response.text)

str

Pour récupérer les 1000 premiers charactères de la page :

In [5]:
response.text[0:1000]

'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2024 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;école de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande école d&#039;ingénieur dans les domaines des transitions numérique, énergétique et environnementale. Classée dans le groupe A, parmi les meilleures écoles d&#039;ingénieur selon le classement de l&#039;Etudiant. Habilitée par la Commission des Titres d&#039;Ingénieur (CTI). Membre de la Conférence des Grandes Ecoles (CGE). " />\n<meta name="viewport" conte

Pour récupérer les headers HTTP de la réponse :

In [6]:
response.headers

{'Date': 'Tue, 19 Nov 2024 13:02:58 GMT', 'Server': 'Apache', 'Content-Language': 'fr', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-UA-Compatible': 'IE=edge', 'X-Content-Type-Options': 'nosniff', 'Content-Length': '16453', 'Content-Type': 'text/html; charset=utf-8', 'X-Varnish': '109904334 107873684', 'Age': '109', 'Via': '1.1 varnish (Varnish/7.1)', 'Accept-Ranges': 'bytes', 'Connection': 'keep-alive'}

On peut modifier les paramètres de la requête et/ou ses headers. On peut par exemple ajouter un UserAgent (identifiant de l'initiateur de la requête) et un timeout de 10 secondes :

In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers, timeout = 10)
response.content[0:1000]

b'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2024 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;\xc3\xa9cole de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande \xc3\xa9cole d&#039;ing\xc3\xa9nieur dans les domaines des transitions num\xc3\xa9rique, \xc3\xa9nerg\xc3\xa9tique et environnementale. Class\xc3\xa9e dans le groupe A, parmi les meilleures \xc3\xa9coles d&#039;ing\xc3\xa9nieur selon le classement de l&#039;Etudiant. Habilit\xc3\xa9e par la Commission des Titres d&#039;Ing\xc3\xa9nieur (CTI). Membr

## Exercice

## Exercice 1

- Créer une classe Python permettant de faire des requêtes HTTP.
- Cette classe doit utiliser toujours le même UserAgent.
- Le TimeOut sera spécifié à chaque appelle avec une valeur par défaut.
- Un mécanisme de retry sera mis en place de façon recursive.

## Exercice 2

- Faire une fonction permettant de supprimer tous les espaces supperflus d'une string
- Faire une fonction qui prend une string html et renvois une string intelligible (enlever les caractères spéciaux,
- Récupérer le domaine en fonction d'un url

In [8]:
#Exercice 1
import requests
from requests.exceptions import RequestException
import re
from html import unescape
from urllib.parse import urlparse

class HTTP:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    def __init__(self, max_retries=3):
        self.max_retries = max_retries
    
    def http(self, url, timeout=10, retries=0):
        try:
            response = requests.get(url, headers=self.headers, timeout=timeout)
            response.raise_for_status()  
            return response.content[:1000]  
        
        except RequestException as e:
          
            if retries < self.max_retries:
                print(f"Échec de la requête, tentative {retries + 1} de {self.max_retries}...")
                return self.http(url, timeout, retries=retries + 1)  
            else:
                print("Nombre maximum de tentatives atteint. Échec de la requête.")
                return None  

http_client = HTTP(max_retries=3)

result = http_client.http("https://www.esiee.fr/")
print(result)

#Exercice 2 : 
   
def No_Space(string) :
    return ' '.join(string.split())

def String_Int(html) :
    text = re.sub(r'<.*?>', '', html)
    text = unescape(text)
    text = re.sub(r'[&<>]', '', text)
    text = No_Space(text)
    return text

def get_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc 

text = " Ceci   est    un  texte   avec des   espaces   superflus. "
print(No_Space(text)) 
html_string = "<p>Ceci est <strong>un texte</strong> avec &amp; des balises HTML.</p>"
print(String_Int(html_string))
url = "https://www.esiee.fr/"
domain = get_domain(url)
print(domain)  # "www.example.com"

b'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2024 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;\xc3\xa9cole de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande \xc3\xa9cole d&#039;ing\xc3\xa9nieur dans les domaines des transitions num\xc3\xa9rique, \xc3\xa9nerg\xc3\xa9tique et environnementale. Class\xc3\xa9e dans le groupe A, parmi les meilleures \xc3\xa9coles d&#039;ing\xc3\xa9nieur selon le classement de l&#039;Etudiant. Habilit\xc3\xa9e par la Commission des Titres d&#039;Ing\xc3\xa9nieur (CTI). Membr

# Exploitation du HTML  

Ici, il faut récupérer le code HTML d'un site web à partir d'une requête. Lorsque vous avez récupéré le texte d'un site il faut le parser. Pour cela, on utilise BeautifulSoup qui permet de transformer la structure HTML en objet Python. Cela permet de récupérer efficacement les données qui nous intéresse.

Pour les webmasters, le blocage le plus souvent mis en place et un blocage sur le User-Agent. Le User-Agent est un paramètre intégré dans la requête HTTP réalisé par le Navigateur pour envoyer au front des informations basiques :

- la version du Navigateur,
- la version de l'OS
- Le type de gestionnaire graphique (Gecko)
- le type de device utilisé

Exemple de User Agent :  

`Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0`

Commençons à utiliser `BeautifulSoup`, il est normalement déjà installé, le cas échéant executez les lignes suivantes : 

In [9]:
import requests
from bs4 import BeautifulSoup

Pour transformer une requête (requests) en objet BeautifulSoup :

In [10]:
response = requests.get(url)
soup = BeautifulSoup(response.text)

Pour trouver tous les liens d'une page, on récupère la balise `a` qui permet de gérer les liens en HTML :

In [12]:
soup.find_all("a")[0:10]

[<a href="/#content">Aller au contenu</a>,
 <a href="/#menu">Aller au menu</a>,
 <a href="/plan-du-site/">Plan du site</a>,
 <a href="/actualites/journees-portes-ouvertes-2024-2025" target="_blank" title="Ouvre une nouvelle fenêtre">Bien choisir son école, c'est aussi la rencontrer ! Rendez-vous à ESIEE Paris le 7 décembre de 13h à 18h pour notre journée portes ouvertes.</a>,
 <a href="/"><img alt="ESIEE PARIS" class="a42-ac-replace-img" src="/typo3conf/ext/esiee_sitepackage/Resources/Public/imgs/svg/logo-esiee.svg"/></a>,
 <a href="/brochures-1">Brochures</a>,
 <a href="/informations/etudiantes-et-etudiants">Espace élèves</a>,
 <a href="/" hreflang="fr-FR" title="Français">
 <span>Fr</span>
 </a>,
 <a href="/en/" hreflang="en-US" title="English">
 <span>En</span>
 </a>,
 <a href="/candidater-1">Candidater</a>]

On peut aussi préciser la classe HTML qu'on veut récupérer :

```python
soup.find_all(class_="<CLASS_NAME>")[0:10]
```

Ici par exemple: 

In [11]:
soup.find_all(class_="toggler")[0:5]

[<button aria-controls="searchbox-header-form" aria-expanded="false" class="toggler">
 <i class="fa-solid fa-magnifying-glass"></i>
 <i class="fa-solid fa-xmark"></i>
 <span class="sr-only">
 <span class="display">Afficher</span><span class="hide">Masquer</span> la recherche
 		</span>
 </button>,
 <button aria-controls="submenu-40" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>L'école</button>,
 <button aria-controls="submenu-563" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>Gouvernance et conseils</button>,
 <button aria-controls="submenu-65" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>Départements d'enseignements et de recherche</button>,
 <button aria-controls="su

Pour récupérer le text sans les balises HTML :

In [12]:
soup.text[0:1000]

"\n\n\n\n\nESIEE Paris, l'école de l'innovation technologique | ESIEE Paris\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAller au contenu\nAller au menu\nPlan du site\n\n\n\n\n\n\n\nBien choisir son école, c'est aussi la rencontrer ! Rendez-vous à ESIEE Paris le 7 décembre de 13h à 18h pour notre journée portes ouvertes.\n\n\n\n\n\nMasquer l'alerte\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBrochuresEspace élèves\n\n\n\nFr\n\n\n\n\nEn\n\n\n\n\n\n\n\n\n\nAfficherMasquer la recherche\r\n\t\t\n\n\n\nSaisissez votre recherche\xa0:\n\nLancer la recherche\n\n\n\nCandidater\n\nAfficherMasquer le menu\n\n\n\n\n\nRetour au menu principalAfficherMasquer le sous menu\xa0: L'écolePourquoi choisir ESIEE Paris ?AfficherMasquer le sous menu\xa0: Gouvernance et conseilsGouvernance et conseilsConseil scientifiqueAfficherMasquer le sous menu\xa0: Départements d'enseignements et de rechercheInformatique et télécommunicationsIngénierie des systèmes cyberphysiquesIngénierie industrielleSanté, énergie et environnem

## Exercice
### Exercice 3

Améliorer la classe développé précédemment.

- Ajouter une méthode pour récupérer l'objet soup d'un url
- Récupérer une liste de User Agent et effectuer une rotation aléatoire sur celui à utiliser
- Utiliser cette classe pour parser une page HTML et récupérer : le titre, tous les H1 (si ils existent), les liens vers les images, les liens sortants vers d'autres sites, et le texte principal.

In [14]:
import requests
from requests.exceptions import RequestException
from bs4 import BeautifulSoup
import random

class HTTP2:
    # Liste de User-Agents pour rotation
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1',
        'Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Mobile Safari/537.36'
    ]
    
    def __init__(self, max_retries=3):
        self.max_retries = max_retries

    def get_headers(self):
        """Retourne un en-tête avec un User-Agent aléatoire."""
        return {
            'User-Agent': random.choice(self.user_agents)
        }

    def http(self, url, timeout=10, retries=0):
        """
        Effectue une requête HTTP GET avec retries.
        """
        try:
            headers = self.get_headers()  # Rotation du User-Agent
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()  # Vérifie si la requête est réussie
            return response.content  # Retourne le contenu brut
        
        except RequestException as e:
            if retries < self.max_retries:
                print(f"Échec de la requête, tentative {retries + 1} de {self.max_retries}...")
                return self.http(url, timeout, retries + 1)
            else:
                print("Nombre maximum de tentatives atteint. Échec de la requête.")
                return None

    def get_soup(self, url, timeout=10):
        """
        Retourne un objet BeautifulSoup pour une URL donnée.
        """
        html_content = self.http(url, timeout)
        if html_content:
            return BeautifulSoup(html_content, 'html.parser')
        else:
            return None

    def parse_page(self, url, timeout=10):
        """
        Analyse une page HTML pour extraire :
        - Le titre
        - Tous les H1
        - Les liens vers les images
        - Les liens sortants
        - Le texte principal
        """
        soup = self.get_soup(url, timeout)
        if not soup:
            return None
        
        # Récupérer le titre de la page
        title = soup.title.string.strip() if soup.title else "Titre non trouvé"

        # Récupérer tous les H1
        h1_tags = [h1.get_text(strip=True) for h1 in soup.find_all('h1')]

        # Récupérer les liens des images
        image_links = [img['src'] for img in soup.find_all('img') if 'src' in img.attrs]

        # Récupérer les liens sortants (URLs externes)
        links = [a['href'] for a in soup.find_all('a', href=True)]
        domain = requests.utils.urlparse(url).netloc
        outgoing_links = [link for link in links if domain not in link]

        # Récupérer le texte principal
        main_text = ' '.join([p.get_text(strip=True) for p in soup.find_all('p')])

        return {
            "title": title,
            "h1_tags": h1_tags,
            "image_links": image_links,
            "outgoing_links": outgoing_links,
            "main_text": main_text
        }


In [15]:
# Instancier la classe
http_client = HTTP2(max_retries=3)

# URL à analyser
url = "https://www.esiee.fr/"

# Analyser la page HTML
parsed_data = http_client.parse_page(url)

# Afficher les résultats
if parsed_data:
    print("Titre :", parsed_data["title"])
    print("H1 Tags :", parsed_data["h1_tags"])
    print("Liens des images :", parsed_data["image_links"])
    print("Liens sortants :", parsed_data["outgoing_links"])
    print("Texte principal :", parsed_data["main_text"])
else:
    print("Impossible d'analyser la page.")

Titre : ESIEE Paris, l'école de l'innovation technologique | ESIEE Paris
H1 Tags : ['']
Liens des images : ['/typo3conf/ext/esiee_sitepackage/Resources/Public/imgs/svg/logo-esiee.svg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/user_upload/Fichiers/image-home/ESIEE-Home-Main-Picture.webp', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/9/5/csm_actu-jpo-1344x840_6889669208.jpg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/0/b/csm_photos-salons-1344x840_472aff3f3e.jpg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/0/3/csm_sandrine-wamy-trophees-des-femmes-de-l_industrie-2024-1344x840_7f464ae034.jpg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/1/6/csm_illustration-forum-esiee-20

# Exploitation des appels d'API



Losque le front du site récupère des données sur une API gérée par le back, un appel d'API est réalisé. Cet appel est recensé dans les appels réseaux. Il est alors possible de re-jouer cet appel pour récupérer à nouveau les données. Il est très facile de récupérer ces appels dans l'onglet Network de la console développeur de Chrome ou FireFox. La console vous permet de copier le code CURL de la requête et vous pouvez ensuite la transformer en code Python depuis le site https://curl.trillworks.com/.

Souvent les APIs sont bloquées avec certains paramètres. L'API vérifie que dans les headers de la requête HTTP ces paramètres sont présents :
* un token généré à la volée avec des protocoles OAuth2 (ou moins développés).
* un referer provenant du site web (la source de la requête), très facile à falsifier.



## Exercice 
### Exercice 4

- Utiliser les informations développées plus haut pour récupérer les premiers résultats d'une recherche d'une requête
sur Google. 

Tips : 

- Ouvrir les outils de développements de Chrome ou Firefox
- Onglet Network
- Fouiller dans les requêtes pour voir à quoi ressemble un appel API Google
- Utilisez beautiful soup pour convertir le contenu de la request en objet et accéder aux balises

In [16]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

class GoogleSearch:
    def __init__(self, max_results=10):
        self.base_url = "https://www.google.com/search"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.max_results = max_results

    def search(self, query):
        """
        Effectue une recherche sur Google et retourne les résultats.
        """
        # Encoder la requête pour qu'elle soit compatible avec une URL
        encoded_query = urllib.parse.quote_plus(query)
        url = f"{self.base_url}?q={encoded_query}&num={self.max_results}"

        # Envoyer une requête GET
        response = requests.get(url, headers=self.headers)
        if response.status_code != 200:
            print(f"Erreur lors de la requête : {response.status_code}")
            return None

        # Utiliser BeautifulSoup pour analyser le contenu HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraire les résultats
        results = []
        for item in soup.select('div.tF2Cxc'):  # Classe contenant un résultat
            title = item.select_one('h3').text if item.select_one('h3') else None
            link = item.select_one('a')['href'] if item.select_one('a') else None
            snippet = item.select_one('.VwiC3b').text if item.select_one('.VwiC3b') else None

            if title and link:
                results.append({
                    'title': title,
                    'link': link,
                    'snippet': snippet
                })

        return results


In [23]:
# Instancier la classe
google_search = GoogleSearch(max_results=5)

# Effectuer une recherche
query = "Rémi Locquette"
results = google_search.search(query)

# Afficher les résultats
if results:
    print("Résultats de recherche Google :")
    for i, result in enumerate(results, 1):
        print(f"{i}. Titre : {result['title']}")
        print(f"   Lien : {result['link']}")
        print(f"   Extrait : {result['snippet']}\n")
else:
    print("Aucun résultat trouvé.")


Résultats de recherche Google :
1. Titre : Rémi LOCQUETTE - Animateur intervenant - TAOS EVENT
   Lien : https://fr.linkedin.com/in/r%C3%A9mi-locquette-0a665425b
   Extrait : j'ai travaillé dans des entreprises de différents domaines : -Stagiaire dans un labo de Calibration -Animateur de groupe (adultes comme enfants)

2. Titre : Rémi Locquette (@remi_locquette)
   Lien : https://www.instagram.com/remi_locquette/
   Extrait : 758 Followers, 791 Following, 12 Posts - Rémi Locquette (@remi_locquette) on Instagram: "Mon objectif dans la vie : l'UTMB"

3. Titre : Classements des coureurs s'appelant Remi Locquette
   Lien : https://www.kikourou.net/resultats/locquette+remi.html
   Extrait : Kikouroù est un site de course à pied, trail, marathon. Vous trouvez des récits, résultats, photos, vidéos de course, un calendrier, un forum... Bonne visite !

4. Titre : Rémi Locquette
   Lien : https://m.facebook.com/remi.locquette.3/
   Extrait : Rémi Locquette is on Facebook. Join Facebook to connec

# Exercice Final  

Exercice Final
Utilisez tout ce que vous avez appris pour récupérer des articles de News avec une catégorie. Il est souvent intéressant de partir des flux RSS pour commencer :

Les données doivent comprendre :
- Le texte important propre
- L'url
- Le domaine
- la catégorie
- Le titre de l'article
- Le titre de la page
- (Facultatif) : les images

Tips : 

- Taper le nom de votre média favoris + RSS (par exemple : https://www.lemonde.fr/rss/)
- Aller dans le DOM de la page 
- Trouver les catégories et les liens vers les articles

In [24]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

class RSSNewsScraper:
    def __init__(self, rss_url):
        self.rss_url = rss_url
    
    def fetch_rss_feed(self):
        """
        Récupère le contenu XML du flux RSS.
        """
        try:
            response = requests.get(self.rss_url)
            response.raise_for_status()
            return response.content
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la récupération du flux RSS : {e}")
            return None
    
    def parse_rss_feed(self, rss_content):
        """
        Analyse le flux RSS et extrait les articles.
        """
        root = ET.fromstring(rss_content)
        articles = []

        for item in root.findall(".//item"):
            title = item.find("title").text
            link = item.find("link").text
            description = item.find("description").text if item.find("description") is not None else ""
            category = item.find("category").text if item.find("category") is not None else "Non catégorisé"
            
            articles.append({
                "title": title,
                "url": link,
                "description": description,
                "category": category,
                "domain": urlparse(link).netloc
            })
        
        return articles
    
    def fetch_article_content(self, url):
        """
        Récupère le contenu principal d'un article (texte principal et images).
        """
        try:
            response = requests.get(url, headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            })
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraire le titre de la page
            page_title = soup.title.string if soup.title else "Titre non trouvé"

            # Extraire le texte principal
            paragraphs = soup.find_all('p')
            main_text = " ".join([p.get_text(strip=True) for p in paragraphs])

            # Extraire les images
            images = [img['src'] for img in soup.find_all('img') if 'src' in img.attrs]

            return {
                "page_title": page_title,
                "main_text": main_text,
                "images": images
            }
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la récupération de l'article : {e}")
            return None
    
    def scrape_news(self):
        """
        Scrape les articles depuis le flux RSS et récupère les détails de chaque article.
        """
        rss_content = self.fetch_rss_feed()
        if not rss_content:
            return None
        
        articles = self.parse_rss_feed(rss_content)
        for article in articles:
            article_content = self.fetch_article_content(article["url"])
            if article_content:
                article.update(article_content)  # Ajouter les détails de l'article
        return articles


In [ ]:
# Exemple avec un flux RSS
rss_url = "https://www.lemonde.fr/rss/en_continu.xml"
scraper = RSSNewsScraper(rss_url)

# Récupérer les articles
articles = scraper.scrape_news()

# Afficher les articles
if articles:
    for i, article in enumerate(articles, 1):
        print(f"Article {i}:")
        print(f"  Titre : {article['title']}")
        print(f"  URL : {article['url']}")
        print(f"  Domaine : {article['domain']}")
        print(f"  Catégorie : {article['category']}")
        print(f"  Titre de la page : {article['page_title']}")
        print(f"  Contenu principal : {article['main_text'][:200]}...")  # Limité à 200 caractères
        print(f"  Images : {article['images']}\n")
else:
    print("Aucun article trouvé.")


ParseError: syntax error: line 1, column 0 (<string>)

In [29]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

class MediapartRSSScraper:
    def __init__(self, rss_url):
        self.rss_url = rss_url
    
    def fetch_rss_feed(self):
        """
        Récupère le contenu XML du flux RSS.
        """
        try:
            response = requests.get(self.rss_url)
            response.raise_for_status()
            return response.content
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la récupération du flux RSS : {e}")
            return None
    
    def parse_rss_feed(self, rss_content):
        """
        Analyse le flux RSS et extrait les articles.
        """
        root = ET.fromstring(rss_content)
        articles = []

        for item in root.findall(".//item"):
            title = item.find("title").text
            link = item.find("link").text
            description = item.find("description").text if item.find("description") is not None else ""
            category = item.find("category").text if item.find("category") is not None else "Non catégorisé"
            
            articles.append({
                "title": title,
                "url": link,
                "description": description,
                "category": category,
                "domain": urlparse(link).netloc
            })
        
        return articles
    
    def fetch_article_content(self, url):
        """
        Récupère le contenu principal d'un article (texte principal et images).
        """
        try:
            response = requests.get(url, headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            })
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraire le titre de la page
            page_title = soup.title.string if soup.title else "Titre non trouvé"

            # Extraire le texte principal
            paragraphs = soup.find_all('p')
            main_text = " ".join([p.get_text(strip=True) for p in paragraphs])

            # Extraire les images
            images = [img['src'] for img in soup.find_all('img') if 'src' in img.attrs]

            return {
                "page_title": page_title,
                "main_text": main_text,
                "images": images
            }
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la récupération de l'article : {e}")
            return None
    
    def scrape_news(self):
        """
        Scrape les articles depuis le flux RSS et récupère les détails de chaque article.
        """
        rss_content = self.fetch_rss_feed()
        if not rss_content:
            return None
        
        articles = self.parse_rss_feed(rss_content)
        for article in articles:
            article_content = self.fetch_article_content(article["url"])
            if article_content:
                article.update(article_content)  # Ajouter les détails de l'article
        return articles


In [30]:
# URL du flux RSS de Mediapart
rss_url = "https://www.mediapart.fr/articles/feed"
scraper = MediapartRSSScraper(rss_url)

# Récupérer les articles
articles = scraper.scrape_news()

# Afficher les articles
if articles:
    for i, article in enumerate(articles, 1):
        print(f"Article {i}:")
        print(f"  Titre : {article['title']}")
        print(f"  URL : {article['url']}")
        print(f"  Domaine : {article['domain']}")
        print(f"  Catégorie : {article['category']}")
        print(f"  Titre de la page : {article['page_title']}")
        print(f"  Contenu principal : {article['main_text'][:200]}...")  # Limité à 200 caractères
        print(f"  Images : {article['images']}\n")
else:
    print("Aucun article trouvé.")


Article 1:
  Titre : Au procès de la rue d’Aubagne, les familles redonnent vie aux disparus des effondrements
  URL : https://www.mediapart.fr/journal/france/191124/au-proces-de-la-rue-d-aubagne-les-familles-redonnent-vie-aux-disparus-des-effondrements
  Domaine : www.mediapart.fr
  Catégorie : France
  Titre de la page : Au procès de la rue d’Aubagne, les familles redonnent vie aux disparus des effondrements | Mediapart
  Contenu principal : Mardi 19 novembre Édition de la mi-journée Newsletters Découvrez notre offre de newsletters L’application Mediapart Découvrez notre application pour mobile & tablette Justice Après les experts, le tri...
  Images : ['https://www.mediapart.fr/assets/build/default/front/nouvelle_formule/images/banner/112x112/banner_newsletter.2df2b340b2fdba47.png', 'https://www.mediapart.fr/assets/build/default/front/nouvelle_formule/images/banner/112x112/banner_mobile.e8ae17b0e8f4f68d.jpg', 'https://www.mediapart.fr/assets/build/default/front/nouvelle_formule/image